In [1]:
# put this just below the imports before calling main
import argparse
parser = argparse.ArgumentParser(description='training')
parser.add_argument('-node', default=None, help='node to use, in format boole-n021')
parser.add_argument('-model_dir', default=None, help='the folder containing the preprocessed data')
parser.add_argument('-gpu', default=0, help='which gpu to use')
opts = parser.parse_args()

In [3]:
node = 'boole-n021'
gpu = '0'

In [21]:
from sigopt import Connection
conn = Connection(client_token="IFAQZABYDOBABXMSZYAWSYKHYSONHNPEACATCSCCIDXDQFLG")

In [31]:
experiment_id = '27604'
# print(conn.experiments(experiment_id).suggestions().fetch(
#     state="open",
# ))

In [55]:
import glob
preprocessed_dir = glob.glob('/home/henrye/projects/E2E/experiments/*' + experiment_id + '*')[0]

In [32]:
suggestion = conn.experiments(experiment_id).suggestions().create()
print(suggestion)

Suggestion({
  "assignments": {
    "x": 0.12237073625468398,
    "y": 0.8270471893339983
  },
  "created": 1509363138,
  "experiment": "27604",
  "fold": null,
  "fold_index": null,
  "id": "12843825",
  "metadata": null,
  "object": "suggestion",
  "state": "open"
})


In [36]:
training_opts = ' '.join('{} {}'.format(key, value) for key, value in suggestion.assignments.items())

In [1]:
import subprocess

In [54]:
preprocessed_dir[0]

'/home/henrye/projects/E2E/experiments/baseline-test-27604-2017_10_29__15_03'

In [84]:
training_output = subprocess.run(['bash',
    '/home/henrye/projects/E2E/bash_training_scripts/start_training_template.sh',
    '-n', node,
    '-g', gpu,
    '-p', preprocessed_dir,
    '-t', training_opts],
              stdout=subprocess.PIPE)
training_output = training_output.stdout.decode()

In [85]:
try:
    checkpoint_dir = re.search('the checkpoint directory is (.*) end transmission', \
                               training_output).group(1)
except AttributeError:
    checkpoint_dir = ''
    print('no checkpoint found? should we quit or skip to next iteration')
#     quit()
print(checkpoint_dir)


/home/henrye/projects/E2E/experiments/baseline-test-27604-2017_10_29__15_03/checkpoints-boole-n021-0-2017_10_29__16_51


In [4]:
checkpoint_dir='/home/henrye/projects/E2E/experiments/baseline-test-27604-2017_10_29__15_03/checkpoints-boole-n021-0-2017_10_29__16_51'
evaluation_opts = ''

In [14]:
evaluation_output_full = subprocess.run(['bash',
    '/home/henrye/projects/E2E/bash_training_scripts/evaluate_model_checkpoint.sh',
    '-n', node,
    '-g', gpu,
    '-c', checkpoint_dir,
    '-e', evaluation_opts],
              stdout=subprocess.PIPE)
evaluation_output = evaluation_output_full.stdout.decode()
# if we want to test it ourselves
print(' '.join(evaluation_output_full.args))

bash /home/henrye/projects/E2E/bash_training_scripts/evaluate_model_checkpoint.sh -n boole-n021 -g 0 -c /home/henrye/projects/E2E/experiments/baseline-test-27604-2017_10_29__15_03/checkpoints-boole-n021-0-2017_10_29__16_51 -e 


In [15]:
# print(evaluation_output)
import re
generated_outputs_directory = re.search('the generated outputs directory is (.*) end transmission', \
                               evaluation_output).group(1)
print(generated_outputs_directory)
# output_scores = re.search('this checkpoint scores are (.*) end transmission', \
#                                evaluation_output).group(1)
# print(output_scores)

/home/henrye/projects/E2E/experiments/baseline-test-27604-2017_10_29__15_03/checkpoints-boole-n021-0-2017_10_29__16_51/generated-outputs-boole-n021-gpu_0-2017_10_30__11_07


In [16]:
import glob
import os
output_scores = glob.glob(os.path.join(generated_outputs_directory, 'dev*'))
output_scores = 

['/home/henrye/projects/E2E/experiments/baseline-test-27604-2017_10_29__15_03/checkpoints-boole-n021-0-2017_10_29__16_51/generated-outputs-boole-n021-gpu_0-2017_10_30__11_07/devset-source.tok.unique-BLEU,0.6537,NIST,7.7660,METEOR,0.4819,ROUGE_L,0.7319,CIDEr,2.3479-checkpoint_e8.pt']

In [21]:
import re
this =['/home/henrye/projects/E2E/experiments/baseline-test-27604-2017_10_29__15_03/checkpoints-boole-n021-0-2017_10_29__16_51/generated-outputs-boole-n021-gpu_0-2017_10_30__11_07/devset-source.tok.unique-BLEU,0.6537,NIST,7.7660,METEOR,0.4819,ROUGE_L,0.7319,CIDEr,2.3479-checkpoint_e8.pt',
      '/home/henrye/projects/E2E/experiments/baseline-test-27604-2017_10_29__15_03/checkpoints-boole-n021-0-2017_10_29__16_51/generated-outputs-boole-n021-gpu_0-2017_10_30__11_07/devset-source.tok.unique-BLEU,0.7537,NIST,7.7660,METEOR,0.4819,ROUGE_L,0.7319,CIDEr,2.3479-checkpoint_e8.pt']
this = sorted(this, reverse=True)
re.search('BLEU,(.*),NIST', this[0]).group(1)


'0.7537'

In [111]:
list_of_strings = ['BLEU,0.7637,NIST,7.7660,METEOR,0.4819,ROUGE_L,0.7319,CIDEr,2.3479',
                   'BLEU,0.5637,NIST,8.7660,METEOR,0.4819,ROUGE_L,0.7319,CIDEr,2.3479',
                  'BLEU,0.6637,NIST,7.7660,METEOR,0.4819,ROUGE_L,0.7319,CIDEr,2.3479',]
sorted(list_of_strings, reverse=True)

['BLEU,0.7637,NIST,7.7660,METEOR,0.4819,ROUGE_L,0.7319,CIDEr,2.3479',
 'BLEU,0.6637,NIST,7.7660,METEOR,0.4819,ROUGE_L,0.7319,CIDEr,2.3479',
 'BLEU,0.5637,NIST,8.7660,METEOR,0.4819,ROUGE_L,0.7319,CIDEr,2.3479']

In [36]:
    suggestion = \
    {
      "assignments": {
        "x": 0.789986936091864,
        "y": 0.03618364763806858
      }
    }

In [39]:
suggestion['assignments'].items()

dict_items([('x', 0.789986936091864), ('y', 0.03618364763806858)])

In [8]:
from collections import defaultdict
this = defaultdict(lambda:'lol')
this['whwhwh']

'lol'

In [15]:
dict_of_list_of_dicts_of_parameters = \
    {
        'baseline':
            [
                {
                    'name': '-word_vec_dim',
                    'type': 'int',
                    'bounds': 
                    {
                        'min': 100.0,
                        'max': 1000.0
                    }
                }
            ]
    }

test_dict = defaultdict(lambda:'lol', dict_of_list_of_dicts_of_parameters)

In [17]:
test_dict['baseline']

[{'bounds': {'max': 1000.0, 'min': 100.0},
  'name': '-word_vec_dim',
  'type': 'int'}]

In [29]:
# Run `pip install sigopt` to download the python API client
from sigopt import Connection
from sigopt.examples import franke_function

In [30]:
conn = Connection(client_token="IFAQZABYDOBABXMSZYAWSYKHYSONHNPEACATCSCCIDXDQFLG")

In [1]:

experiment = conn.experiments().create(
    name='Franke Optimization (Python)',
    parameters=[
        dict(name='x', type='double', bounds=dict(min=0.0, max=1.0)),
        dict(name='y', type='double', bounds=dict(min=0.0, max=1.0)),
    ],
)
print("Created experiment: https://sigopt.com/experiment/" + experiment.id)

# Evaluate your model with the suggested parameter assignments
# Franke function - http://www.sfu.ca/~ssurjano/franke2d.html
def evaluate_model(assignments):
    return franke_function(assignments['x'], assignments['y'])

# Run the Optimization Loop between 10x - 20x the number of parameters
for _ in range(30):
    suggestion = conn.experiments(experiment.id).suggestions().create()
    value = evaluate_model(suggestion.assignments)
    conn.experiments(experiment.id).observations().create(
        suggestion=suggestion.id,
        value=value,
    )


Created experiment: https://sigopt.com/experiment/27604


In [4]:
suggestion = conn.experiments(27604).suggestions().create()

In [7]:
suggestion

Suggestion({
  "assignments": {
    "x": 0.9439681426019584,
    "y": 0.17154101101444252
  },
  "created": 1509270406,
  "experiment": "27604",
  "fold": null,
  "fold_index": null,
  "id": "12833147",
  "metadata": null,
  "object": "suggestion",
  "state": "open"
})

we probably don't have to create the experiment from scratch necessarily

In [4]:
print("You can track your experiment at https://sigopt.com/experiment/{0}".format(experiment.id))

You can track your experiment at https://sigopt.com/experiment/27604


In [5]:
experiment

Experiment({
  "client": "6526",
  "conditionals": [],
  "created": 1509026516,
  "development": false,
  "folds": null,
  "id": "27604",
  "linear_constraints": [],
  "metadata": null,
  "metric": {
    "name": null,
    "object": "metric",
    "value_baseline": null
  },
  "metrics": [
    {
      "name": null,
      "object": "metric",
      "value_baseline": null
    }
  ],
  "name": "Franke Optimization (Python)",
  "num_solutions": 1,
  "object": "experiment",
  "observation_budget": null,
  "parallel_bandwidth": 1,
  "parameters": [
    {
      "bounds": {
        "max": 1.0,
        "min": 0.0,
        "object": "bounds"
      },
      "categorical_values": null,
      "conditions": {},
      "default_value": null,
      "name": "x",
      "object": "parameter",
      "precision": null,
      "tunable": true,
      "type": "double"
    },
    {
      "bounds": {
        "max": 1.0,
        "min": 0.0,
        "object": "bounds"
      },
      "categorical_values": null,
      "

In [7]:
dir(conn)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'clients',
 'experiments',
 'impl',
 'set_api_url',
 'set_proxies',
 'set_verify_ssl_certs']